In [79]:
import torch
from cgan_code_2 import Generator  # 假设你有一个定义好的 Discriminator 类
import json
import random
import math

In [80]:
NPRINT_LINE_LEN = 114
# NPRINT_REAL_WIDTH = 50*8
NPRINT_REAL_WIDTH = 22*8
LABEL_DICT = {'facebook': 0, 'skype': 1, 'aim': 2, 'email': 3, 'voipbuster': 4, 'hangouts': 5, 'youtube': 6, 'sftp': 7, 'icq': 8,  'ftps': 9, 'vimeo': 10, 'spotify': 11, 'netflix': 12, 'bittorrent': 13}
# LABEL_DICT = {'facebook': 0, 'skype': 1}
BYTES_LOG_MEAN = 5.961416185547372
BYTES_LOG_STD = 1.4073518072041993
PACKETS_LOG_MEAN = 1.5394048328338192
PACKETS_LOG_STD = 1.1067132728539024

SEQ_DIM = 3
MAX_PKT_LEN = 1500
MAX_TIME = 1000
MAX_PORT = 65536
MAX_SEQ_LEN = 16

label_dim = len(LABEL_DICT) 
image_dim = (1, NPRINT_REAL_WIDTH, NPRINT_REAL_WIDTH)  # 生成单通道图像
# metadata_dim = 2+16+2  # 元数据维度为2
noise_dim = 100  # 噪声维度
batch_size = 64
epochs = 200

In [81]:
bins_data = {}
with open('./bins.json', 'r') as f_bin:
    bins_data = json.load(f_bin)
    
port_intervals = bins_data['port']['intervals']
pkt_len_intervals = []
for bins in bins_data['packet_len']:
    pkt_len_intervals.append(bins['intervals'])
time_intervals = []
for bins in bins_data['time']:
    time_intervals.append(bins['intervals'])

In [82]:
# 创建模型实例（与保存时的模型架构相同）
generator = Generator(label_dim, noise_dim, SEQ_DIM, MAX_SEQ_LEN, 'cpu')

# 加载模型权重
checkpoint = torch.load('./save_all/generator_250.pth')  # 加载保存的权重字典
generator.load_state_dict(checkpoint)  # 将权重字典加载到模型中

# 切换到评估模式
generator.eval()

/tmp/ipykernel_206500/1327513028.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('./save_all/generator_250.pth')  # 加载保存的权重字典


Generator(
  (label_fc): Sequential(
    (0): Linear(in_features=14, out_features=128, bias=True)
    (1): ReLU(inplace=True)
  )
  (length_fc): Sequential(
    (0): Linear(in_features=16, out_features=128, bias=True)
    (1): ReLU(inplace=True)
  )
  (noise_fc): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): ReLU(inplace=True)
  )
  (combine_fc): Sequential(
    (0): Linear(in_features=384, out_features=512, bias=True)
    (1): ReLU(inplace=True)
  )
  (lstm): LSTM(512, 512, num_layers=4, batch_first=True)
  (output_layer): Sequential(
    (0): Linear(in_features=512, out_features=3, bias=True)
    (1): Tanh()
  )
)

In [83]:
labels = torch.zeros(batch_size, label_dim).to('cpu')  # 初始化为全 0
labels[:, 1] = 1  # 将每个样本的第一个值设置为 1，形成 one-hot 编码   
noise = torch.randn(batch_size, noise_dim).to('cpu') 
lengths = torch.zeros(batch_size).to('cpu')
lengths.fill_(2)
seqs = torch.clamp(generator(labels, noise, lengths),-1,1)

In [84]:
id = 0
for seq in seqs.tolist():
    for i in range(min(int(lengths.tolist()[id]),MAX_SEQ_LEN)):
        [time,pkt_len,dport] = seq[i]
        # print(time,pkt_len,dport)
        
        
        time_id = math.floor((time + 1)/2*len(time_intervals[i]))
        pkt_len_id = math.floor((pkt_len + 1)/2*len(pkt_len_intervals[i]))
        dport_id = math.floor((dport + 1)/2*len(port_intervals))
        # print(time_id,pkt_len_id,dport_id)
        
        time = round(random.uniform(time_intervals[i][time_id][0], time_intervals[i][time_id][1]),2)
        pkt_len = round(random.uniform(pkt_len_intervals[i][pkt_len_id][0], pkt_len_intervals[i][pkt_len_id][1]))
        dport = round(random.uniform(port_intervals[dport_id][0],port_intervals[dport_id][1]))

        # time = round(time)
        # pkt_len = round(pkt_len)
        # dport = round(dport)
        
        # if sign > 0:
        #     pkt_len = - pkt_len
        print(time,pkt_len,dport)
    id+=1
    print()

0.0 60 21
0.0 -263 53

0.0 54 21315
0.09 52 24530

0.0 60 21
0.0 -268 53

0.0 55 40030
0.01 -64 39975

0.0 62 21
0.01 -406 21

0.0 50 138
0.01 -61 465

0.0 60 22
0.0 -242 53

0.0 61 36897
0.01 -60 39942

0.0 60 22
0.0 -396 21

0.0 41 20779
0.12 57 23514

0.0 41 15918
0.36 64 17697

0.0 68 53
0.38 -335 21

0.0 61 22
0.0 -246 53

0.0 60 22
0.0 -276 21

0.0 48 43259
0.01 -61 39942

0.0 62 35672
0.07 -54 38870

0.0 50 45582
0.01 -63 39975

0.0 63 22
0.01 -400 21

0.0 50 123
0.27 -91 138

0.0 52 43206
0.01 -64 39975

0.0 50 45411
0.01 -63 39975

0.0 61 22
0.0 -242 53

0.0 52 40406
0.01 -64 39975

0.0 66 53
0.2 -374 21

0.0 46 40028
0.41 -58 39032

0.0 60 21
0.0 -244 53

0.0 60 22
0.0 -244 53

0.0 52 41263
0.01 -62 39975

0.0 60 22
0.0 -389 21

0.0 63 35916
0.34 -60 39700

0.0 56 24011
0.41 30 29810

0.0 61 22
0.0 -253 22

0.0 64 53
0.01 -396 22

0.0 68 53
0.36 -335 21

0.0 51 43206
0.01 -64 39975

0.0 46 35394
0.35 -48 35916

0.0 63 35916
0.37 -60 39713

0.0 72 465
0.01 -117 123

0.0 59 390